<a href="https://colab.research.google.com/github/osram90/whisper/blob/main/UltimoGoodMusicGen_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

  Cloning https://github.com/facebookresearch/audiocraft to /tmp/pip-install-xx1zb1li/audiocraft_0b6131ea42d84386926ffe917638918b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/audiocraft /tmp/pip-install-xx1zb1li/audiocraft_0b6131ea42d84386926ffe917638918b
  Resolved https://github.com/facebookresearch/audiocraft to commit adf0b04a4452f171970028fcf80f101dd5e26e19
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.5 MB/s eta 0:00:00
  Prepari

In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
import torch

In [ ]:
model = musicgen.MusicGen.get_pretrained('stereo-large', device='cuda')


In [ ]:
import numpy as np
import random
import torch

# Set the seed for reproducibility
seed = 42

# Python built-in random module
random.seed(seed)

# NumPy
np.random.seed(seed)

# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# TensorFlow (if needed)
import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
model.set_generation_params(duration=90)
res = model.generate([
    'background sound using binaural frequencies, with appearances in modular moments of the ancestral instrument diyeridu',
],
    progress=True)
display_audio(res, 32000)

In [ ]:
# @title **MP3**
from pydub import AudioSegment
import soundfile as sf
import io
import numpy as np

# Suponiendo que display_audio está definido o importado correctamente
# display_audio(res[0], sample_rate=32000)

def save_audio_to_mp3(audio_data, filename, sample_rate):
    # Transferir tensor a la memoria de la CPU si es necesario
    if audio_data.device.type == 'cuda':
        audio_data = audio_data.cpu()

    # Reshape the audio data if it's not in the expected 2D format
    if audio_data.ndim == 3:
        # Assuming mono audio, we take the first channel
        audio_data = audio_data[0, 0, :]
    elif audio_data.ndim == 1:
        # Si es un audio mono ya en 1D, no se necesita cambio
        pass
    elif audio_data.ndim == 2:
        # Si es estéreo, tomar un canal o mezclarlos
        audio_data = audio_data[0, :]  # Ejemplo para tomar solo el primer canal

    with io.BytesIO() as f:
        # Write the audio data as WAV
        sf.write(f, audio_data.numpy(), sample_rate, format='WAV')
        f.seek(0)
        # Read the audio data as WAV
        audio_segment = AudioSegment.from_file(f, format='WAV')
        # Export as MP3
        audio_segment.export(filename, format='mp3')

# Uso de la función actualizada
# Asegúrese de que 'res[0]' sea un tensor de PyTorch con los datos de audio
save_audio_to_mp3(res[0], 'output_audio.mp3', 32000)
import base64
from IPython.display import HTML

def create_download_link(filename):
    with open(filename, 'rb') as f:
        data = f.read()
        b64 = base64.b64encode(data)
        payload = b64.decode()

    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{filename}</a>'
    html = html.format(payload=payload,filename=filename)
    return HTML(html)

# Crear enlace de descarga para el archivo MP3
create_download_link('output_audio.mp3')


In [ ]:
# @title **FLAC**
from pydub import AudioSegment
import soundfile as sf
import io

def save_audio_to_flac(audio_data, filename, sample_rate):
    # Transferir tensor a la memoria de la CPU si es necesario
    if audio_data.device.type == 'cuda':
        audio_data = audio_data.cpu()

    # Manejar diferentes dimensiones de audio_data
    if audio_data.ndim == 3:
        audio_data = audio_data[0, 0, :]
    elif audio_data.ndim == 2:
        audio_data = audio_data[0, :]  # Tomar solo el primer canal para audio estéreo

    with io.BytesIO() as f:
        # Escribir los datos de audio como WAV
        sf.write(f, audio_data.numpy(), sample_rate, format='WAV')
        f.seek(0)
        # Leer los datos de audio como WAV
        audio_segment = AudioSegment.from_file(f, format='WAV')
        # Exportar como FLAC
        audio_segment.export(filename, format='flac')

# Uso de la función para guardar en formato FLAC
save_audio_to_flac(res[0], 'output_audio.flac', 32000)

import base64
from IPython.display import HTML

def create_download_link(filename):
    with open(filename, 'rb') as f:
        data = f.read()
        b64 = base64.b64encode(data)
        payload = b64.decode()

    html = '<a download="{filename}" href="data:audio/flac;base64,{payload}" target="_blank">{filename}</a>'
    html = html.format(payload=payload,filename=filename)
    return HTML(html)

# Crear enlace de descarga para el archivo FLAC
create_download_link('output_audio.flac')



See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
